# Computing aggregation metrics for the jobs

Here i'm using the full processed Marconi100 dataset

## Reading all job files

In [2]:
import glob
import pandas as pd

DATASET_PATH = "/home/dancarastan/Documentos/exadata_job_energy_profiles/"

jobfiles_list = glob.glob(DATASET_PATH+"*"+"/plugin=job_table"+"/metric=job_info_marconi100"+"/a_0_filter123*")

print(len(jobfiles_list))
#df_jobs = pd.concat([pd.read_csv(jobfile) for jobfile in jobfiles_list]).reset_index(drop=True)


18


## Computing the metrics for each job file

Max, mean, median, std, interquartile range (iqr), inter 10-90 percentile range

In [3]:
lst_result = []
current_month = 0

In [5]:
from scipy.stats import iqr
import numpy as np

#lst_result = []

jobfile_start = 1
jobfile_end = 2

for jobfile in jobfiles_list[jobfile_start:jobfile_end]:
    df_jobs = pd.read_csv(jobfile).reset_index(drop=True)
    month = pd.to_datetime(df_jobs["submit_time"])[0].month
    if month != current_month:
        powerfiles_list = glob.glob(DATASET_PATH+"year_month=22-0"+str(month)+"/plugin=ipmi_pub"+"/metric=total_power"+"/a_0_filter123*")
        df_power = pd.concat([pd.read_csv(powerfile) for powerfile in powerfiles_list]).reset_index(drop=True)
        df_power['node'] = pd.to_numeric(df_power['node'])
        df_power['value'] = pd.to_numeric(df_power['value'])
        #print(df_power)
    powertrace_jobs = [df_power[df_power["job_id"]==x]['value'].values for x in df_jobs['job_id'].values]
    df_jobs["total_power_max_watts"] = [x.max() for x in powertrace_jobs]
    df_jobs["total_power_mean_watts"] = [x.mean() for x in powertrace_jobs]
    df_jobs["total_power_median_watts"] = [np.median(x) for x in powertrace_jobs]
    df_jobs["total_power_std_watts"] = [x.std() for x in powertrace_jobs]
    df_jobs["total_power_iqr_watts"] = [iqr(x, rng=(25, 75)) for x in powertrace_jobs]
    df_jobs["total_power_1090range_watts"] = [iqr(x, rng=(10, 90)) for x in powertrace_jobs]

    lst_result.append(df_jobs)
    current_month = month
    #break
#df_result = pd.concat(lst_result).reset_index(drop=True)
#df_result

/tmp/ipykernel_16035/757584804.py:10: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_jobs = pd.read_csv(jobfile).reset_index(drop=True)


KeyboardInterrupt: 

In [25]:
#df_jobs.to_csv(DATASET_PATH+"Marconi100_all_jobs.csv.tar.gz")

## Reading all energy files

In [1]:
import glob
import pandas as pd

DATASET_PATH = "/home/dancarastan/Documentos/exadata_job_energy_profiles/"

powerfiles_list = glob.glob(DATASET_PATH+"*"+"/plugin=ipmi_pub"+"/metric=total_power"+"/a_0_filter123*")

df_power = pd.concat([pd.read_csv(powerfile) for powerfile in powerfiles_list]).reset_index(drop=True)

KeyboardInterrupt: 